In [9]:
import joblib
import pandas as pd

# === 1. Charger le modèle et le vectorizer ===
model = joblib.load("naive_bayes_model.joblib")        # Ton modèle IA
vectorizer = joblib.load("tfid.joblib")            # Ton TfidfVectorizer ou CountVectorizer

# === 2. Charger ou simuler des emails ===
# Remplace ceci par la lecture depuis ta boîte mail IMAP si tu l'as déjà fait
emails = [
    "Get cheap meds now, no prescription needed!",
    "Bonjour Thomas, votre rendez-vous est confirmé pour demain.",
    "YOU WON a million dollars, click to claim your prize!",
    "Merci pour votre paiement, voici votre reçu.",
]

# === 3. Transformer les emails en vecteurs ===
X_emails = vectorizer.transform(emails)

# === 4. Prédiction des classes ===
predictions = model.predict(X_emails)

# === 5. Affichage des résultats ===
results = pd.DataFrame({
    "Email": emails,
    "Classe prédite": predictions
})

print("\n=== Résultat de la classification des emails ===")
print(results.to_string(index=False))



=== Résultat de la classification des emails ===
                                                      Email  Classe prédite
                Get cheap meds now, no prescription needed!               0
Bonjour Thomas, votre rendez-vous est confirmé pour demain.               0
      YOU WON a million dollars, click to claim your prize!               1
               Merci pour votre paiement, voici votre reçu.               0


In [3]:
import imaplib
import email
import os
import joblib
import datetime

# Chargement du modèle IA et du vectorizer
model = joblib.load("naive_bayes_model.joblib")
vectorizer = joblib.load("tfid.joblib")

# Connexion à Gmail
EMAIL = "thomasondo76@gmail.com"
PASSWORD = "z b e o u r a w d o s j l x c s"
IMAP_SERVER = "imap.gmail.com"

# Dossiers locaux pour enregistrer les mails
os.makedirs("SPAM", exist_ok=True)
os.makedirs("HAM", exist_ok=True)

def analyser_boite(boite="inbox", prefix="inbox"):
    print(f"\n🔍 Analyse des mails depuis la boîte : {boite.upper()}")
    mail = imaplib.IMAP4_SSL(IMAP_SERVER)
    mail.login(EMAIL, PASSWORD)
    mail.select(boite)

    # Récupérer les IDs des mails
    status, messages = mail.search(None, "ALL")
    mail_ids = messages[0].split()

    # Parcours des 10 derniers mails
    for num in mail_ids[-10:]:
        _, data = mail.fetch(num, "(RFC822)")
        msg = email.message_from_bytes(data[0][1])
        subject = msg["subject"]
        sender = msg["from"]

        # Extraction du corps
        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == "text/plain":
                    body = part.get_payload(decode=True).decode(errors="ignore")
                    break
        else:
            body = msg.get_payload(decode=True).decode(errors="ignore")

        # Prédiction IA
        content = subject + " " + body
        vect_content = vectorizer.transform([content])
        prediction = model.predict(vect_content)[0]

        # Formatage du nom de fichier
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{prefix}_{timestamp}.txt"
        result_folder = "SPAM" if prediction == 1 else "HAM"
        path = os.path.join(result_folder, filename)

        # Sauvegarde locale
        with open(path, "w", encoding="utf-8") as f:
            f.write(f"From: {sender}\nSubject: {subject}\n\n{body}")

        print(f"[{result_folder}] Mail archivé sous {path}")

# === Lancer l’analyse sur l’Inbox
analyser_boite("inbox", prefix="inbox")

# === Lancer l’analyse sur le dossier SPAM
analyser_boite("[Gmail]/Spam", prefix="spam")



🔍 Analyse des mails depuis la boîte : INBOX
[HAM] Mail archivé sous HAM\inbox_20250414_194143.txt
[HAM] Mail archivé sous HAM\inbox_20250414_194146.txt
[HAM] Mail archivé sous HAM\inbox_20250414_194148.txt
[HAM] Mail archivé sous HAM\inbox_20250414_194149.txt
[HAM] Mail archivé sous HAM\inbox_20250414_194151.txt
[HAM] Mail archivé sous HAM\inbox_20250414_194153.txt
[HAM] Mail archivé sous HAM\inbox_20250414_194154.txt
[HAM] Mail archivé sous HAM\inbox_20250414_194158.txt
[HAM] Mail archivé sous HAM\inbox_20250414_194159.txt
[HAM] Mail archivé sous HAM\inbox_20250414_194200.txt

🔍 Analyse des mails depuis la boîte : [GMAIL]/SPAM
[HAM] Mail archivé sous HAM\spam_20250414_194215.txt
